In [24]:
import os
import pandas as pd
import win32com.client
from datetime import datetime


def processar_aniversariantes():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes.xlsx"
    NOME_ABA = "Sheet1"

    MES_CARDS = 3  # Março (pasta dos cartões)
    MES_PROCESSAR = 7  # Julho (mês do evento)
    ANO = datetime.now().year

    MESES = {
        1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
        5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
        9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
    }

    PASTA_CARDS = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes - Março"
    CAMINHO_IMAGEM = r"C:\imagens\marca.png"
    EMAIL_DESTINO = "davi.araujo@grendene.com.br"

    if not os.path.exists(PASTA_CARDS):
        print(f"[AVISO] Pasta para o mês {MESES[MES_CARDS]} não encontrada: {PASTA_CARDS}")
        return

    print(f"[INFO] Usando cartões de {MESES[MES_CARDS]} para criar eventos em {MESES[MES_PROCESSAR]}")

    try:
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA, header=0)
        df.columns = ['Nome Funcionário', 'Dia', 'Mês', 'MÊS_NUMERO']

        df_mes = df[df['MÊS_NUMERO'] == MES_CARDS]

        if df_mes.empty:
            print(f"Não há aniversariantes no mês {MES_CARDS}")
            return

        outlook = win32com.client.Dispatch("Outlook.Application")
        namespace = outlook.GetNamespace("MAPI")
        calendar_folder = namespace.GetDefaultFolder(9)  # Calendar

        for dia, grupo in df_mes.groupby('Dia', sort=True):
            try:
                nomes = grupo['Nome Funcionário'].tolist()
                nomes_texto = "; ".join(nomes)

                print(f"\n[PROCESSANDO] Aniversariantes do dia {dia:02d}: {nomes_texto}")

                data_evento = datetime(ANO, MES_PROCESSAR, int(dia), 8, 0)

                # === CRIAR COMPROMISSO ===
                appointment = outlook.CreateItem(1)  # 1 = olAppointmentItem

                if len(nomes) == 1:
                    titulo = f"Aniversário de {nomes[0]}"
                else:
                    titulo = f"Aniversário de {' e '.join(nomes)}"

                appointment.Subject = titulo
                appointment.Start = data_evento
                appointment.Duration = 30
                appointment.AllDayEvent = False
                appointment.ReminderSet = True
                appointment.ReminderMinutesBeforeStart = 1440
                appointment.BusyStatus = 0  # Livre

                corpo_texto = f"""
Olá!

Segue lembrete de aniversário dos colaboradores:
{nomes_texto}.

ATENÇÃO: No dia do aniversário, enviar a imagem em anexo na equipe de celebrações no Teams.

Data: {dia:02d}/{MES_PROCESSAR:02d}/{ANO}
Horário: 08:00 às 08:30

Atenciosamente,
[ A marca está anexada a este compromisso ]
"""

                appointment.Body = corpo_texto

                # Anexar imagem da marca no compromisso, se existir
                if os.path.exists(CAMINHO_IMAGEM):
                    appointment.Attachments.Add(CAMINHO_IMAGEM)

                # Adicionar participante
                recipient = appointment.Recipients.Add(EMAIL_DESTINO)
                recipient.Type = 1  # Obrigatório
                appointment.MeetingStatus = 1  # Reunião

                if appointment.Recipients.ResolveAll():
                    appointment.Save()

                    # Anexar cartões dos aniversariantes
                    anexos = 0
                    arquivos = os.listdir(PASTA_CARDS)
                    for nome in nomes:
                        nome_lower = nome.lower()
                        for arquivo in arquivos:
                            if nome_lower in arquivo.lower():
                                caminho_completo = os.path.join(PASTA_CARDS, arquivo)
                                appointment.Attachments.Add(caminho_completo)
                                anexos += 1
                                break

                    appointment.Send()
                    print(f"[OK] Evento criado e enviado para {EMAIL_DESTINO} com {anexos} anexos.")
                else:
                    print(f"[ERRO] Não foi possível resolver o email {EMAIL_DESTINO}.")

                # === ENVIAR EMAIL COM IMAGEM ===
                mail = outlook.CreateItem(0)  # 0 = olMailItem

                mail.Subject = f"Lembrete de Aniversário - {dia:02d}/{MES_PROCESSAR:02d}"
                mail.To = EMAIL_DESTINO

                # Adiciona a imagem da marca como recurso embutido (inline)
                if os.path.exists(CAMINHO_IMAGEM):
                    attachment = mail.Attachments.Add(CAMINHO_IMAGEM)
                    attachment.PropertyAccessor.SetProperty(
                        "http://schemas.microsoft.com/mapi/proptag/0x3712001F", "marca"
                    )
                    img_tag = '<img src="cid:marca" style="width:300px;">'
                else:
                    img_tag = ""

                corpo_email = f"""
<html>
<body>
<p>Olá!</p>

<p>Segue lembrete de aniversário dos colaboradores:<br>
<b>{nomes_texto}</b>.</p>

<p><b>ATENÇÃO:</b> No dia do aniversário, enviar a imagem em anexo na equipe de celebrações no Teams.</p>

<p>Atenciosamente,</p>
{img_tag}
</body>
</html>
"""

                mail.HTMLBody = corpo_email

                mail.Send()
                print(f"[OK] E-mail enviado para {EMAIL_DESTINO} com imagem da marca.")

            except Exception as e:
                print(f"[ERRO] Falha no dia {dia:02d}: {str(e)}")
                continue

        print("\n[OK] Processamento concluído!")

    except Exception as e:
        print(f"[ERRO] Falha geral na automação: {str(e)}")


if __name__ == "__main__":
    processar_aniversariantes()


[INFO] Usando cartões de Março para criar eventos em Julho

[PROCESSANDO] Aniversariantes do dia 01: DENER DAVI SOUSA LIMA; MARTIM LUIS CREMA
[OK] Evento criado e enviado para davi.araujo@grendene.com.br com 2 anexos.
[OK] E-mail enviado para davi.araujo@grendene.com.br com imagem da marca.

[PROCESSANDO] Aniversariantes do dia 02: AFRANIO BIN; CAMILA DE AZEVEDO OLIVEIRA
[OK] Evento criado e enviado para davi.araujo@grendene.com.br com 2 anexos.
[OK] E-mail enviado para davi.araujo@grendene.com.br com imagem da marca.

[PROCESSANDO] Aniversariantes do dia 04: GABRIEL CAMARGO
[OK] Evento criado e enviado para davi.araujo@grendene.com.br com 1 anexos.
[OK] E-mail enviado para davi.araujo@grendene.com.br com imagem da marca.

[PROCESSANDO] Aniversariantes do dia 08: JOHNATAS FELIX ZUZA; PRISCILA MERZONI
[OK] Evento criado e enviado para davi.araujo@grendene.com.br com 2 anexos.
[OK] E-mail enviado para davi.araujo@grendene.com.br com imagem da marca.

[PROCESSANDO] Aniversariantes do dia 

In [ ]:
mail = outlook.CreateItem(0)  # 0 = olMailItem
mail.Subject = "Teste de imagem inline"
mail.To = EMAIL_DESTINO

if os.path.exists(CAMINHO_IMAGEM):
    attachment = mail.Attachments.Add(CAMINHO_IMAGEM)
    attachment.PropertyAccessor.SetProperty(
        "http://schemas.microsoft.com/mapi/proptag/0x3712001F",
        "marca@aniversario"
    )
    img_tag = '<img src="cid:marca@aniversario" style="width:300px;">'
else:
    img_tag = ""

corpo_email = f"""
<html>
<body>
<p>Olá!</p>
<p>Segue a imagem embutida:</p>
{img_tag}
</body>
</html>
"""

mail.HTMLBody = corpo_email
mail.Send()
print("Email com imagem inline enviado!")



[TESTE] Processando aniversariantes do dia 01: DENER DAVI SOUSA LIMA; MARTIM LUIS CREMA
[TESTE] Evento salvo e email enviado para davi.araujo@grendene.com.br para o dia 01

[TESTE] Processando aniversariantes do dia 02: AFRANIO BIN; CAMILA DE AZEVEDO OLIVEIRA
[TESTE] Evento salvo e email enviado para davi.araujo@grendene.com.br para o dia 02

[TESTE] Processamento concluído para os 2 primeiros aniversariantes.


In [23]:
import os
import pandas as pd
import win32com.client
from datetime import datetime

def processar_aniversariantes():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes.xlsx"
    NOME_ABA = "Sheet1"

    MES_CARDS = 3  # Março (pasta onde estão os cartões)
    MES_PROCESSAR = 7  # Julho (mês para criar eventos)
    ANO = datetime.now().year

    MESES = {
        1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
        5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
        9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
    }

    PASTA_CARDS = rf"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes - {MESES[MES_CARDS]}"
    CAMINHO_MARCA = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Marca_GovTi.png"
    EMAIL_DESTINO = "davi.araujo@grendene.com.br"

    if not os.path.exists(PASTA_CARDS):
        print(f"[AVISO] Pasta para o mês {MESES[MES_CARDS]} não encontrada: {PASTA_CARDS}")
        return

    print(f"[INFO] Usando cartões de {MESES[MES_CARDS]} para criar eventos em {MESES[MES_PROCESSAR]}")

    try:
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA, header=0)
        df.columns = ['Nome Funcionário', 'Dia', 'Mês', 'MÊS_NUMERO']

        df_mes = df[df['MÊS_NUMERO'] == MES_CARDS]

        if df_mes.empty:
            print(f"Não há aniversariantes no mês {MES_CARDS}")
            return

        # Inicializar Outlook uma vez só
        outlook = win32com.client.Dispatch("Outlook.Application")
        namespace = outlook.GetNamespace("MAPI")
        calendar_folder = namespace.GetDefaultFolder(9)  # Calendário

        for dia, grupo in df_mes.groupby('Dia', sort=True):
            try:
                nomes = grupo['Nome Funcionário'].tolist()
                nomes_texto = "; ".join(nomes)

                print(f"\n[PROCESSANDO] Aniversariantes do dia {dia:02d}: {nomes_texto}")

                data_evento = datetime(ANO, MES_PROCESSAR, int(dia), 8, 0)

                # Criar evento no calendário
                appointment = outlook.CreateItem(1)  # olAppointmentItem

                if len(nomes) == 1:
                    titulo = f"Aniversário de {nomes[0]}"
                else:
                    titulo = f"Aniversário de {' e '.join(nomes)}"

                appointment.Subject = titulo
                appointment.Start = data_evento
                appointment.Duration = 30
                appointment.AllDayEvent = False
                appointment.ReminderSet = True
                appointment.ReminderMinutesBeforeStart = 1440
                appointment.BusyStatus = 0

                # Corpo do evento no calendário - texto simples sem imagem
                corpo_evento = f"""
Olá!

Segue lembrete de aniversário dos colaboradores:
{nomes_texto}.

ATENÇÃO: No dia do aniversário, enviar a imagem em anexo na equipe de celebrações no Teams.

Data: {dia:02d}/{MES_PROCESSAR:02d}/{ANO}
Horário: 08:00 às 08:30

Atenciosamente,
GovTI - Equipe de Automação
"""
                appointment.Body = corpo_evento
                appointment.Save()

                # Criar email para enviar lembrete com imagem embutida
                mail = outlook.CreateItem(0)  # olMailItem
                mail.To = EMAIL_DESTINO
                mail.Subject = titulo

                # Imagem embutida (cid)
                if os.path.exists(CAMINHO_MARCA):
                    attachment = mail.Attachments.Add(CAMINHO_MARCA)
                    attachment.PropertyAccessor.SetProperty(
                        "http://schemas.microsoft.com/mapi/proptag/0x3712001F", "marca"
                    )
                    img_tag = '<img src="cid:marca" style="width:300px;">'
                else:
                    img_tag = ""

                # Corpo do email com HTML + imagem inline (rodapé com marca)
                corpo_email_html = f"""
                <html>
                <body>
                <p>Olá!</p>
                <p>Segue lembrete de aniversário dos colaboradores:<br>
                {nomes_texto}.</p>
                <p><b>ATENÇÃO:</b> No dia do aniversário, enviar a imagem em anexo na equipe de celebrações no Teams.</p>
                <p>Data: {dia:02d}/{MES_PROCESSAR:02d}/{ANO}<br>
                Horário: 08:00 às 08:30</p>
                <p>Atenciosamente,</p>
                <div style="background-color:#5C2D91; color:#ffffff; padding:10px; width:fit-content; font-family:Arial, sans-serif; font-weight:bold;">
                    GovTI | Grendene
                </div>
                <br>
                {img_tag}
                </body>
                </html>
                """

                mail.HTMLBody = corpo_email_html
                mail.Send()  # Envia direto. Troque por mail.Display() se quiser revisar antes.

                print(f"[OK] Evento criado e email enviado para {EMAIL_DESTINO}.")

            except Exception as e:
                print(f"[ERRO] Falha no dia {dia:02d}: {str(e)}")
                continue

        print("\n[OK] Processamento concluído!")

    except Exception as e:
        print(f"[ERRO] Falha geral na automação: {str(e)}")

if __name__ == "__main__":
    processar_aniversariantes()


[INFO] Usando cartões de Março para criar eventos em Julho

[PROCESSANDO] Aniversariantes do dia 01: DENER DAVI SOUSA LIMA; MARTIM LUIS CREMA
[OK] Evento criado e email enviado para davi.araujo@grendene.com.br.

[PROCESSANDO] Aniversariantes do dia 02: AFRANIO BIN; CAMILA DE AZEVEDO OLIVEIRA
[OK] Evento criado e email enviado para davi.araujo@grendene.com.br.

[PROCESSANDO] Aniversariantes do dia 04: GABRIEL CAMARGO
[OK] Evento criado e email enviado para davi.araujo@grendene.com.br.

[PROCESSANDO] Aniversariantes do dia 08: JOHNATAS FELIX ZUZA; PRISCILA MERZONI
[OK] Evento criado e email enviado para davi.araujo@grendene.com.br.

[PROCESSANDO] Aniversariantes do dia 11: ELIZETE APARECIDA DE MATOS
[OK] Evento criado e email enviado para davi.araujo@grendene.com.br.

[PROCESSANDO] Aniversariantes do dia 12: NELSON DE ALMEIDA MACHADO BATISTA
[OK] Evento criado e email enviado para davi.araujo@grendene.com.br.

[PROCESSANDO] Aniversariantes do dia 15: ARIEL POSTAL
[OK] Evento criado e ema